In [1]:
%load_ext autoreload

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
%autoreload
from trainer import *
from SupContrastLoss import SupConLoss
from RankingLoss import MultipleNegativesSymmetricRankingLoss

ModuleNotFoundError: No module named 'datadreamer'

In [4]:
def train_datadreamer_ta2(fold, output_folder, used_loss, luar_model_path='./rrivera1849'):
    with DataDreamer(output_folder):
        dataset = DataSource(
            "Train Data",
            data=train_data_generator,
            total_num_rows=train_num_rows,
        )
        dev_dataset = DataSource(
            "Dev Data",
            data=dev_data_generator,
            total_num_rows=dev_num_rows,
        )
    
        trainer = get_luar_trainer()(
            "LUAR Trainer",
            model_name=luar_model_path,
            peft_config=LoraConfig(),
            trust_remote_code=True,
            device='cuda:0',
            dtype="bfloat16",
            force=True,
        )

        loss = SupConLoss if used_loss=='SupConLoss' else losses.MultipleNegativesSymmetricRankingLoss

        if used_loss == 'SupConLoss':
            trainer.train_with_labeled_pairs(
                train_anchors=dataset.output["anchors"],
                train_others= dataset.output["others"],
                train_labels=dataset.output["labels"],
                validation_anchors=dev_dataset.output["anchors"],
                validation_others=dev_dataset.output["others"],
                validation_labels=dev_dataset.output["labels"],
                epochs=25,    
                batch_size=128,
                loss=loss,
                learning_rate=0.0005,
                early_stopping_threshold=0.001,
                early_stopping_patience=5,
                accelerator_config={
                    "dispatch_batches": False,
                },
                callbacks=[EpochTrackerCallback()]
            )
        else:
            trainer.train_with_positive_pairs(
                train_anchors=dataset.output["anchors"],
                train_positives=dataset.output["positives"],
                validation_anchors=dev_dataset.output["anchors"],
                validation_positives=dev_dataset.output["positives"],
                epochs=25,    
                batch_size=100,
                loss=loss,
                learning_rate=0.0005,
                early_stopping_threshold=0.001,
                early_stopping_patience=5,
                accelerator_config={
                    "dispatch_batches": False,
                },
                callbacks=[EpochTrackerCallback()]
            )

### Train original TA2 system

In [5]:
output_path = '/mnt/swordfish-pool2/milad/datadreamer-ta2/'
luar_model_path = '/mnt/swordfish-pool2/milad/rrivera1849'

In [6]:
fold = "/mnt/swordfish-pool2/milad/hiatus-data/train-test-dev split/{split}/Official Query-candidate format/"

In [7]:
train_num_rows, train_data_generator = get_data_generator(fold, "cross_genre_all", "train", split_percent=ast.literal_eval("None"))
dev_num_rows, dev_data_generator = get_data_generator(fold, "cross_genre_all", "dev", split_percent=ast.literal_eval("None"))

Dataset Statistics: cross_genre_all train None 575 4142
Dataset Statistics: cross_genre_all dev None 79 635


In [8]:
train_datadreamer_ta2(fold, output_path + '/original_ta2_model', 'MultipleNegativesSymmetricRankingLoss', luar_model_path=luar_model_path)

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: /mnt/swordfish-pool2/milad/datadreamer-ta2//original_ta2_model
[ 🤖 DataDreamer 💤 ] Step 'Train Data' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Train Data' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Trainer 'LUAR Trainer' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Trainer 'LUAR Trainer' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Anchors' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Anchors' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Positives' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Positives' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / 

### Train original TA2 system with Supervised Contrastive Loss:

In [ ]:
train_num_rows, train_data_generator = get_data_generator_for_supervised_contrastive_learning(fold, "cross_genre_all", "train", split_percent=ast.literal_eval("None"))
dev_num_rows, dev_data_generator = get_data_generator_for_supervised_contrastive_learning(fold, "cross_genre_all", "dev", split_percent=ast.literal_eval("None"))

In [ ]:
train_datadreamer_ta2(fold, output_path + '/supcon_ta2_model', 'SupConLoss', luar_model_path=luar_model_path)